Папка на GDrive:

https://drive.google.com/drive/folders/1FeUwrVfxp09xAifiMngwtfrRZ8nhtj8T?usp=sharing


In [53]:
from os import listdir

USE_GDRIVE = False


if USE_GDRIVE:
    from google.colab import drive
    drive.mount("/home/GDrive")
    DATA_PATH = f"/home/GDrive/MyDrive/made_2021_fraud_project/data"
else:
    DATA_PATH = "../../data"

GPS_PATH = "../../data/all_gps.csv"
ACCEL_PATH = "../../data/all_accel.csv"

listdir(DATA_PATH)

['unlabeled',
 'good_data',
 'all_gps.csv',
 'all_accel.csv',
 'drivers_with_gps_and_sl_problems',
 'rides_of_drivers.csv']

In [54]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import folium
from haversine import haversine
from os import listdir


%matplotlib inline

In [55]:
def describe(df):
    display(pd.concat((df.dtypes, df.describe(datetime_is_numeric=True).T), axis=1))
    display(df.shape)

In [56]:
df_gps = pd.read_csv(GPS_PATH, parse_dates=[0, 3])
describe(df_gps)

,0,count,mean,min,25%,50%,75%,max,std
time,datetime64[ns],5050292,2021-04-06 10:23:00.788006400,2020-10-25 03:59:58,2021-01-16 06:40:41.750000128,2021-04-07 17:01:10.500000,2021-06-22 15:23:15.499999744,2021-08-28 02:47:58,NaN
lat,float64,5.05029e+06,55.743,42.98,55.6674,55.7496,55.8242,82.4343,0.171211
lon,float64,5.05029e+06,37.6021,26.1013,37.5009,37.5848,37.7032,135.46,0.401377
gps_time,datetime64[ns],5050292,2021-04-06 10:22:58.275877632,2020-10-25 03:59:56,2021-01-16 06:40:38.749999872,2021-04-07 17:01:10.500000,2021-06-22 15:23:15.499999744,2021-08-28 02:47:58,NaN
driver_hash,int64,5.05029e+06,3.03558e+17,-9.21858e+18,-4.06958e+18,1.02237e+17,4.79299e+18,9.20647e+18,5.27957e+18
fraud,float64,1.06611e+06,0.100867,0,0,0,0,1,0.301152


(5050292, 6)

In [57]:
df_accel = pd.read_csv(ACCEL_PATH, parse_dates=[0])
describe(df_accel)

,0,count,mean,min,25%,50%,75%,max,std
time,datetime64[ns],2172989,2020-11-22 15:27:12.714645504,2015-03-01 05:46:42.832594,2020-12-18 06:53:34.205120,2021-03-18 16:54:00.549220096,2021-06-12 11:02:18.664983040,2021-08-29 00:03:16.287344,NaN
x,float64,2.17299e+06,0.368934,-71.1054,-0.371101,0.184359,0.785298,65.1529,2.36914
y,float64,2.17299e+06,7.6569,-74.7916,7.63271,8.85376,9.50019,55.2294,3.4034
z,float64,2.17299e+06,3.83295,-65.5062,2.01402,3.69992,5.35365,76.4756,2.73072
lat,float64,2.17299e+06,55.7482,55.3163,55.6768,55.7512,55.8214,82.4343,0.16893
lon,float64,2.17299e+06,37.601,36.3143,37.5016,37.5863,37.686,108.856,0.376061
driver_hash,int64,2.17299e+06,-1.12657e+17,-9.21858e+18,-4.47912e+18,-4.14168e+17,4.43147e+18,9.20647e+18,5.3536e+18
fraud,float64,335525,0.315374,0,0,0,1,1,0.464666


(2172989, 8)

In [58]:
def map_driver(driver_gps, driver_accel):
    center = (driver_gps["lat"].mean(), driver_gps["lon"].mean())
    color = driver_accel.head(1)["fraud"].replace({1:"red", 0:"green", np.nan:"yellow"}).values[0]
    m = folium.Map(location=center, zoom_start=10, legend=True)
    fg = folium.FeatureGroup(name=f"gps").add_to(m)
    folium.PolyLine(driver_gps[["lat", "lon"]], color='blue', weight=8, label="tracks", opacity=0.8).add_to(fg)
    fg = folium.FeatureGroup(name=f"accel").add_to(m)
    folium.PolyLine(driver_accel[["lat", "lon"]], color=color, weight=6, label="accel", opacity=0.8).add_to(fg)
    folium.map.LayerControl('topleft', collapsed= False).add_to(m)
    return m

In [117]:
FORCE_FRAUD = 1

if not FORCE_FRAUD is None:
    cur_driver = np.random.choice(df_gps.loc[df_gps["fraud"] == FORCE_FRAUD, "driver_hash"].unique())
else:
    cur_driver = np.random.choice(df_gps["driver_hash"].unique())

# пример водителя в большим количесвтом сматченных точек
# cur_driver = -6646726793274239750

cur_driver

-8915411586673655243

In [121]:
driver_gps = df_gps[df_gps["driver_hash"] == cur_driver]
driver_accel = df_accel[df_accel["driver_hash"] == cur_driver]

display(
    f"Current Driver:{cur_driver}",
    driver_gps.describe(datetime_is_numeric=True).T,
    driver_accel.describe(datetime_is_numeric=True).T
)

map_driver(driver_gps, driver_accel)

'Current Driver:-8915411586673655243'

,count,mean,min,25%,50%,75%,max,std
time,55,2021-01-21 20:01:47.345454336,2021-01-21 18:42:53,2021-01-21 18:43:20.500000,2021-01-21 19:07:22,2021-01-21 21:12:13.500000,2021-01-22 00:21:50,NaN
lat,55,55.5891,55.3931,55.6323,55.6324,55.6332,55.699,0.0985462
lon,55,37.6195,37.4714,37.4716,37.4745,37.8936,37.9211,0.204278
gps_time,55,2021-01-21 20:01:47.181818112,2021-01-21 18:42:53,2021-01-21 18:43:20.500000,2021-01-21 19:07:21,2021-01-21 21:12:13.500000,2021-01-22 00:21:50,NaN
driver_hash,55,-8.91541e+18,-8.91541e+18,-8.91541e+18,-8.91541e+18,-8.91541e+18,-8.91541e+18,2066.88
fraud,55,1,1,1,1,1,1,0


,count,mean,min,25%,50%,75%,max,std
time,2690,2021-01-19 19:45:01.783707136,2021-01-19 18:02:51.749420,2021-01-19 19:04:06.599343360,2021-01-19 19:04:11.842354432,2021-01-19 20:54:34.183333120,2021-01-19 21:20:52.420308,NaN
x,2690,-0.0492244,-4.30956,-0.557849,-0.0227449,0.591368,4.78122,1.06609
y,2690,8.8008,3.28963,8.5066,8.84897,9.18895,14.0707,0.85941
z,2690,4.219,-5.1619,3.86963,4.18746,4.50828,12.0931,1.01239
lat,2690,55.7661,55.6704,55.7032,55.7032,55.8312,55.8993,0.0790564
lon,2690,37.5977,37.4632,37.5908,37.6264,37.6265,37.6586,0.0579295
driver_hash,2690,-8.91541e+18,-8.91541e+18,-8.91541e+18,-8.91541e+18,-8.91541e+18,-8.91541e+18,1024.19
fraud,2690,1,1,1,1,1,1,0


In [119]:
df_merged = pd.merge_asof(driver_gps, driver_accel,
              left_on="gps_time",
              right_on="time",
              direction="nearest",
              tolerance=pd.Timedelta("60s"),
              suffixes=("", "_r"),
              allow_exact_matches=False)

df_matched = df_merged.dropna(thresh=7).copy() \
    .drop(columns=["driver_hash_r", "fraud_r"]) \
    .rename(columns={"time_r": "accel_time", "lat_r": "accel_lat", "lon_r": "accel_lon"})

display(f"{cur_driver} Matched: {df_matched.shape[0]} of {df_merged.shape[0]}")
if df_matched.shape[0] > 0:
    describe(df_matched)

'-8915411586673655243 Matched: 0 of 55'

In [62]:
cols_gps = ["driver_hash", "fraud", "time", "lat", "lon", "gps_time"]
df_matched_gps = df_matched[cols_gps]

cols_accel = ["driver_hash", "fraud", "accel_time", "accel_lat", "accel_lon", "x", "y", "z"]
df_matched_accel = df_matched[cols_accel] \
    .rename(columns={c:c.replace("accel_", "") for c in cols_accel})

map_driver(df_matched_gps, df_matched_accel)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
gps_time_diff = df_matched["gps_time"].diff()
gps_time_diff.fillna(gps_time_diff.mean(), inplace=True)
display(gps_time_diff.round("5min").value_counts())

accel_time_diff = df_matched["accel_time"].diff()
accel_time_diff.fillna(accel_time_diff.diff().mean(), inplace=True)
display(accel_time_diff.round("5min").value_counts())

In [ ]:
df_matched["track"] = gps_time_diff.round("10min").cumsum().dt.seconds
number_of_tracks = len(df_matched["track"].unique())
display(f"Number of tracks: {number_of_tracks}")

df_matched = df_matched[["driver_hash", "fraud", "track"] + cols_gps[2:] + cols_accel[2:]].copy()
describe(df_matched)

In [ ]:
df_matched

In [ ]:
center = (df_matched["lat"].mean(), df_matched["lon"].mean())
m = folium.Map(location=center, zoom_start=10, legend=True)

color = driver_accel.head(1)["fraud"].replace({1:"red", 0:"green", np.nan:"yellow"}).values[0]

for track in df_matched["track"].unique():
    fg = folium.FeatureGroup(name=f"track{track}")
    gps_track = df_matched[df_matched["track"] == track]
    folium.PolyLine(gps_track[["lat", "lon"]], color="blue", weight=8, label="accel", opacity=0.8).add_to(fg)
    accel_track = df_matched[df_matched["track"] == track]
    folium.PolyLine(accel_track[["accel_lat", "accel_lon"]], color=color, weight=6, label="tracks", opacity=0.8).add_to(fg)
    fg.add_to(m)

folium.map.LayerControl('topleft', collapsed=False).add_to(m)
m

### Вывод - такая разбивка на поездки выглядит вполне правдоподобной